In [ ]:
# CSE 144 - Assignment 3
# RNN for Natural Language Processing
# Ishika Pol

# transformer.py provided by TA was used as base code

# Import the necessary libraries
!pip install transformers
!pip install datasets

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("yelp_review_full")
reviews = dataset["train"]["text"]
labels = dataset["train"]["label"]

# Split the dataset into train and test sets
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Define the RNN model
class SentimentClassifier(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super(SentimentClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = model
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.embedding(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        _, hidden = self.rnn(last_hidden_state)
        logits = self.fc(hidden.squeeze(0))
        return logits

# Preprocess the input text
def preprocess(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    return inputs['input_ids'], inputs['attention_mask']

# Convert labels to tensor
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
hidden_size = model.config.hidden_size
num_labels = dataset["train"].features["label"].num_classes
model = SentimentClassifier(hidden_size, num_labels)
model = model.to(device)

# Set hyperparameters
learning_rate = 1e-4
batch_size = 32
num_epochs = 5

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Create data loaders
train_loader = DataLoader(list(zip(train_reviews, train_labels)), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(list(zip(test_reviews, test_labels)), batch_size=batch_size)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = preprocess(inputs)
        inputs = [input.to(device) for input in inputs]
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(*inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        predicted_labels = []
        true_labels = []
        for inputs, labels in test_loader:
            inputs = preprocess(inputs)
            inputs = [input.to(device) for input in inputs]
            labels = labels.to(device)

            logits = model(*inputs)
            _, predicted = torch.max(logits, 1)

            predicted_labels.extend(predicted.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    # Print evaluation metrics
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print()

# Model interpretation
top_k = 10  # Number of top words to analyze
word_importances = []
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = preprocess(inputs)
        inputs = [input.to(device) for input in inputs]

        logits = model(*inputs)
        probabilities = F.softmax(logits, dim=1)

        _, top_indices = torch.topk(probabilities, k=top_k, dim=1)
        top_words = [tokenizer.decode(indices, skip_special_tokens=True) for indices in top_indices]
        word_importances.extend(top_words)

# Print important words
print("Important words contributing to positive sentiment:")
print(word_importances[:top_k])
print()
print("Important words contributing to negative sentiment:")
print(word_importances[-top_k:])
print()

# Model comparison with logistic regression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Preprocess the text for logistic regression
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(train_reviews)
test_features = vectorizer.transform(test_reviews)

# Train logistic regression model
logreg = LogisticRegression()
logreg.fit(train_features, train_labels)

# Evaluate logistic regression model
logreg_predictions = logreg.predict(test_features)
logreg_accuracy = accuracy_score(test_labels, logreg_predictions)
logreg_precision = precision_score(test_labels, logreg_predictions, average='weighted')
logreg_recall = recall_score(test_labels, logreg_predictions, average='weighted')
logreg_f1 = f1_score(test_labels, logreg_predictions, average='weighted')

# Print logistic regression results
print("Logistic Regression Results:")
print(f"Accuracy: {logreg_accuracy:.4f}")
print(f"Precision: {logreg_precision:.4f}")
print(f"Recall: {logreg_recall:.4f}")
print(f"F1-Score: {logreg_f1:.4f}")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.9 MB/s eta 0:00:00


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
